# Netflix Analysis

## <u>Introduction</u>

Our family Netflix account has been shared between 3 profiles, my brother Chase, my mother Sharleen, and myself for over a decade (2013-2024). In that time, we have all watched hundreds of movies and TV shows of different genres. My analysis will show how similar and different we are as a family when it comes to our taste and watching patterns while we are on Netflix.

To determine this, I will be answering 5 questions:

> 1. What years did each profile watch the most Netflix in?
>     
> 2. Which devices does each profile watch on the most?
>     
> 3. What are the average watching periods over each day of the month across all years?
>     
> 4. What are the most popular titles for shows and movies by profile?
>     
> 5. What is the amount of time spent watching each genre by profile and category?
>

\# Data Sources

Provide info about the source of the data for your mart and links to any supportings

\# Steps to simple mart creation

\-- the following is performed in t-sql against a sql server instance. MS Sequel Server 2022 was used for this project.. any other info here...

Intro to Schemas

### <u>Schema Creation</u>

- <span style="color: var(--vscode-foreground);">stg</span>
- <span style="color: var(--vscode-foreground);">dim&nbsp;</span>   
- <span style="color: var(--vscode-foreground);">fact</span>
- <span style="color: var(--vscode-foreground);">vw</span>

  

```
USE Netflix_DW;
GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'stg' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA stg AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'fact' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA fact AUTHORIZATION dbo;'
END


IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name =  'vw'  )
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA vw AUTHORIZATION dbo;'
END

```

Intro to Dimensions

### <u>Dimension Creation</u>

\- Tables Creation:

\- dim.Genres

\-dim.Calendar

\-dim.t3....

fact.Viewing

```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Calendar')
BEGIN
	CREATE TABLE dim.Calendar(
       PK_CalendarDate DATE NOT NULL,
       Year INT NOT NULL,
       Month TINYINT NOT NULL,
       MonthName NVARCHAR(10) NULL,
       Day TINYINT NOT NULL,
	   DayName NVARCHAR(10) NULL,
	   Weekend NVARCHAR(3) NULL,
	   WeekofYear TINYINT NOT NULL
       );

	ALTER TABLE dim.Calendar
	ADD CONSTRAINT PK_Date PRIMARY KEY(PK_CalendarDate);

END
```

```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Devices')
BEGIN
	CREATE TABLE dim.Devices(
	   Device_ID TINYINT NOT NULL,
	   Device_Type NVARCHAR(50) NULL,
	   Times_Device_Used SMALLINT NOT NULL,
	   Device_Usage NVARCHAR(20) NULL
       );

	ALTER TABLE dim.Devices
	ADD CONSTRAINT PK_Devices PRIMARY KEY(Device_ID);

END
```

```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Genres')
BEGIN
	CREATE TABLE dim.Genres(
	   Genre_ID TINYINT NOT NULL,
	   Genre NVARCHAR(25) NOT NULL
       );

	ALTER TABLE dim.Genres
	ADD CONSTRAINT PK_Genre PRIMARY KEY(Genre_ID);

END
```

```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Profiles')
BEGIN
	CREATE TABLE dim.Profiles(
	   Profile_ID TINYINT NOT NULL,
	   Profile_Name NVARCHAR(10) NOT NULL
       );

	ALTER TABLE dim.Profiles
	ADD CONSTRAINT PK_Profile PRIMARY KEY(Profile_ID);

	ALTER TABLE dim.Profiles
	ADD CONSTRAINT UC_Profile UNIQUE (Profile_Name);


END
```

```
--Dim tables to form new dim

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Shows')
BEGIN
    CREATE TABLE dim.Shows(
       ShowID smallint NOT NULL,
       Show_Name nvarchar(100) NOT NULL,
       Genre nvarchar(25) NOT NULL,
       );

END

```
```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Movies')
BEGIN
    CREATE TABLE dim.Movies(
       MovieID smallint NOT NULL,
       Movie_Name nvarchar(100) NOT NULL,
       Genre nvarchar(25) NOT NULL,
       );

END
```

```
--New dim

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Titles')
BEGIN
    CREATE TABLE dim.Titles (
       PK_MovieShow_ID INT IDENTITY(1,1) NOT NULL,
       Title VARCHAR(100) NOT NULL,
       Genre NVARCHAR(25) NOT NULL,
	   Title_ID SMALLINT NULL,
	   [Type] NVARCHAR(6) NULL,
       );

	ALTER TABLE dim.Titles
	ADD CONSTRAINT NetflixID PRIMARY KEY(PK_MovieShow_ID);

END
```

```
-- Fact tables
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'fact' AND TABLE_NAME = 'NetflixShows')
BEGIN
    CREATE TABLE fact.NetflixShows (
        PK_Shows_ID INT IDENTITY(1,1) NOT NULL,
        [Date] DATE NOT NULL,
        Show_Name NVARCHAR(100) NOT NULL,
        Profile_ID TINYINT NOT NULL,
        Show_ID SMALLINT NOT NULL,
        Genre_ID TINYINT NOT NULL,
        Device_ID TINYINT NOT NULL,
        Total_Duration FLOAT NOT NULL
        );
END

```
```
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'fact' AND TABLE_NAME = 'NetflixMovies')
BEGIN
	CREATE TABLE fact.NetflixMovies (
        PK_Movies_ID INT IDENTITY(1,1) NOT NULL,
        [Date] DATE NOT NULL,
        Movie_Name NVARCHAR(100) NOT NULL,
        Profile_ID TINYINT NOT NULL,
        Movie_ID SMALLINT NOT NULL,
        Genre_ID TINYINT NOT NULL,
        Device_ID TINYINT NOT NULL,
        Total_Duration FLOAT NOT NULL
    );
END
```

```
--New fact
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'fact' AND TABLE_NAME = 'Netflix')
BEGIN
    CREATE TABLE fact.Netflix (
        PK_Netflix_ID INT IDENTITY(1,1) NOT NULL,
        [Date] DATE NOT NULL,
        Title NVARCHAR(100) NOT NULL,
        Profile_ID TINYINT NOT NULL,
        Title_ID SMALLINT NOT NULL,
        Genre_ID TINYINT NOT NULL,
        Device_ID TINYINT NOT NULL,
        Total_Duration FLOAT NOT NULL
);


    ALTER TABLE fact.Netflix
    ADD CONSTRAINT PK_Netflix PRIMARY KEY (PK_Netflix_ID);

    ALTER TABLE fact.Netflix
    ADD CONSTRAINT FK_NETFLIXtoCAL
    FOREIGN KEY ([Date])
    REFERENCES dim.Calendar(PK_CalendarDate);

    ALTER TABLE fact.Netflix
    ADD CONSTRAINT FK_NETFLIXtoPROFILE
    FOREIGN KEY (Profile_ID)
    REFERENCES dim.Profiles(Profile_ID);

    ALTER TABLE fact.Netflix
    ADD CONSTRAINT FK_NETFLIXtoDEVICE
    FOREIGN KEY (Device_ID)
    REFERENCES dim.Devices(Device_ID);

END

```

ERD EX

\-- Insert link 

  

\-- Insert image itself